In [11]:
import numpy as np
from PIL import Image
path = "/Users/nole/Desktop/D-NeRF dataset/hellwarrior/train/r_000.png"
image = Image.open(path)
im_data = np.array(image.convert("RGBA"))
norm_data = im_data / 255.0
mask = norm_data[..., 3:4]
print(mask.shape)

(800, 800, 1)


In [1]:
import os
import numpy as np
from PIL import Image
import math
def readWaymoCameras(path, start_time, end_time, cameras):
    cam_infos = []
    ego_dir = os.path.join(path, "ego_pose")
    extrinsic_dir = os.path.join(path, "extrinsics")
    intrinsic_dir = os.path.join(path, "intrinsics")
    image_dir = os.path.join(path, "image")

    for t in range(start_time, end_time+1):
        for cam in cameras:
            ego = np.load(os.path.join(ego_dir, f"{t:03d}.txt"))
            intrinsic = np.loadtxt(os.path.join(intrinsic_dir, f"{cam}.txt"))
            extrinsic = np.load(os.path.join(extrinsic_dir, f"{cam}.txt"))
            image_file = os.path.join(image_dir, f"{t:03d}_{cam}.png")
            img = Image.open(image_file)
            frame_time = (t - start_time) / (end_time - start_time)
            fx, fy, cx, cy = intrinsic[0], intrinsic[1], intrinsic[2], intrinsic[3]
            fovx = 2 * math.atan(cx / fx)
            fovy = 2 * math.atan(cy / fy)
            




def readWaymoSceneInfo(path, start_time, end_time, cameras):
    """
    Params:
        path: including scene id
    Returns:

    """
    print("Reading Waymo data")
    train_cam_infos = readWaymoCameras(path, start_time, end_time, cameras)

